<a href="https://colab.research.google.com/github/muhajirakbarhsb/NLP_class_2023/blob/main/Week_Class_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Week Class 5

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('gpt_clean_review.csv')
df

,at,content,score,userName,contentp,contentp_clean,text_length
0,2023-09-24 14:03:39,Aplikasi ini sangat tidak bagus saya sangatla ...,1,Qinnaya,aplikasi ini sangat tidak bagus saya sangatla ...,aplikasi sangat bagus sangatla marah saya hapu...,56
1,2023-09-24 12:41:29,sangat berguna sekali ya teman teman,5,Ananta Pebrianto,sangat berguna sekali ya teman teman,sangat sekali teman teman,25
2,2023-09-24 12:04:00,Its just so useful for me,5,Dokkan JP,its just so useful for me,it is just so useful for me,27
3,2023-09-24 10:25:59,membuat pintar umat manusia,5,DafiUwU GamingXD,membuat pintar umat manusia,buat pintar umat manusia,24
4,2023-09-24 10:04:36,nicee,5,Dean F,nicee,nice,4
...,...,...,...,...,...,...,...
195,2023-09-12 19:21:14,good,5,danu prabowo,good,good,4
196,2023-09-12 18:15:12,saya amat sangat bahagia telah menemukan ChatG...,5,KHARISMA PUTRI WIBOWO,saya amat sangat bahagia telah menemukan chatg...,amat sangat bahagia temu chatgpt rupa temu lua...,98
197,2023-09-12 15:27:48,Mengecewakan. Ga bisa diedit.,2,Edy Rahman,mengecewakan ga bisa diedit,kecewa bisa edit,16
198,2023-09-12 14:52:51,sungguh sangat membantu jawabannya layan Tepat...,5,Fery Printing,sungguh sangat membantu jawabannya layan tepat...,sungguh sangat bantu jawab layan tepat sayang ...,126


In [3]:
df['sentiment_first_scheme'] = df['score'].apply(lambda x: 'negative' if x in [1, 2] else 'positive')

# Sentiment analysis based on the second scheme
df['sentiment_second_scheme'] = df['score'].apply(lambda x: 'negative' if x in [1, 2] else ('neutral' if x == 3 else 'positive'))

In [4]:
df.sentiment_second_scheme.value_counts()

sentiment_second_scheme
positive    182
negative     13
neutral       5
Name: count, dtype: int64

In [5]:
df.sentiment_first_scheme.value_counts()

sentiment_first_scheme
positive    187
negative     13
Name: count, dtype: int64

### Classification Process

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [7]:
df = df[['contentp_clean', 'sentiment_first_scheme', 'score']]
missing_values = df['contentp_clean'].isna().sum()
if missing_values > 0:
    df['contentp_clean'].fillna('', inplace=True)


C:\Users\Alharidt\AppData\Local\Temp\ipykernel_3368\3942706650.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['contentp_clean'].fillna('', inplace=True)


In [8]:
X = df['contentp_clean']  # Text data
y_binary = (df['sentiment_first_scheme'] == 'positive').astype(int)  # Binary labels (1 for positive, 0 for negative)
y_multi = df['score']  # Multi-class labels (original star ratings)

X_train, X_test, y_binary_train, y_binary_test, y_multi_train, y_multi_test = train_test_split(
    X, y_binary, y_multi, test_size=0.2, random_state=42)

In [9]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [10]:
X_train_tfidf

<160x415 sparse matrix of type '<class 'numpy.float64'>'
	with 858 stored elements in Compressed Sparse Row format>

In [11]:
X_train_tfidf_dense = X_train_tfidf.toarray()

# Print the TF-IDF representation
print("TF-IDF representation of X_train:")
print(X_train_tfidf_dense)

TF-IDF representation of X_train:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
binary_classifier = LogisticRegression()
binary_classifier.fit(X_train_tfidf, y_binary_train)
binary_predictions = binary_classifier.predict(X_test_tfidf)

In [13]:
print("Binary Classification Report:")
print(classification_report(y_binary_test, binary_predictions))

Binary Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.95      1.00      0.97        38

    accuracy                           0.95        40
   macro avg       0.47      0.50      0.49        40
weighted avg       0.90      0.95      0.93        40



C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [14]:
#!pip install xgboost

In [15]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [16]:
# Binary Classification with Random Forest and GridSearchCV
rf_classifier = RandomForestClassifier()
param_grid_rf = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20]}
grid_search_rf = GridSearchCV(rf_classifier, param_grid_rf, cv=3, scoring='accuracy')
grid_search_rf.fit(X_train_tfidf, y_binary_train)
best_rf_model = grid_search_rf.best_estimator_
binary_predictions_rf = best_rf_model.predict(X_test_tfidf)

# Binary Classification with XGBoost and GridSearchCV
xgb_classifier = XGBClassifier()
param_grid_xgb = {'n_estimators': [100, 200, 300], 'max_depth': [3, 4, 5], 'learning_rate': [0.01, 0.1, 0.2]}
grid_search_xgb = GridSearchCV(xgb_classifier, param_grid_xgb, cv=3, scoring='accuracy')
grid_search_xgb.fit(X_train_tfidf, y_binary_train)
best_xgb_model = grid_search_xgb.best_estimator_
binary_predictions_xgb = best_xgb_model.predict(X_test_tfidf)

# Evaluate binary classification model (Random Forest)
print("Binary Classification Report (Random Forest):")
print(classification_report(y_binary_test, binary_predictions_rf))

# Evaluate binary classification model (XGBoost)
print("Binary Classification Report (XGBoost):")
print(classification_report(y_binary_test, binary_predictions_xgb))

C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Alharidt\AppData\Roaming\Python\Pyt

Binary Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.95      0.95      0.95        38

    accuracy                           0.90        40
   macro avg       0.47      0.47      0.47        40
weighted avg       0.90      0.90      0.90        40

Binary Classification Report (XGBoost):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.95      1.00      0.97        38

    accuracy                           0.95        40
   macro avg       0.47      0.50      0.49        40
weighted avg       0.90      0.95      0.93        40



C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [17]:
def predict_sentiment(text):
    # Preprocess the input text (e.g., remove punctuation, lowercase, etc.)
    preprocessed_text = text.lower()  # You can add more pre-processing steps as needed

    # Vectorize the preprocessed text using the TF-IDF vectorizer
    text_vectorized = tfidf_vectorizer.transform([preprocessed_text])

    # Predict sentiment using all three models
    rf_prediction = best_rf_model.predict(text_vectorized)
    xgb_prediction = best_xgb_model.predict(text_vectorized)
    logistic_prediction = binary_classifier.predict(text_vectorized)

    return {
        "Random Forest Prediction": "positive" if rf_prediction[0] == 1 else "negative",
        "XGBoost Prediction": "positive" if xgb_prediction[0] == 1 else "negative",
        "Logistic Regression Prediction": "positive" if logistic_prediction[0] == 1 else "negative",
    }

# Example usage:
input_text = "telkomsel tidak jelas membuat produk, programmernya gaji buta"
sentiment_prediction = predict_sentiment(input_text)
print(sentiment_prediction)

{'Random Forest Prediction': 'positive', 'XGBoost Prediction': 'positive', 'Logistic Regression Prediction': 'positive'}


In [18]:
# Example usage:
input_text = "produknya oke, karyawannya juga ganteng dan tampan"
sentiment_prediction = predict_sentiment(input_text)
print(sentiment_prediction)

{'Random Forest Prediction': 'positive', 'XGBoost Prediction': 'positive', 'Logistic Regression Prediction': 'positive'}


## MultiClass

In [19]:
df

,contentp_clean,sentiment_first_scheme,score
0,aplikasi sangat bagus sangatla marah saya hapu...,negative,1
1,sangat sekali teman teman,positive,5
2,it is just so useful for me,positive,5
3,buat pintar umat manusia,positive,5
4,nice,positive,5
...,...,...,...
195,good,positive,5
196,amat sangat bahagia temu chatgpt rupa temu lua...,positive,5
197,kecewa bisa edit,negative,2
198,sungguh sangat bantu jawab layan tepat sayang ...,positive,5


In [20]:
df

,contentp_clean,sentiment_first_scheme,score
0,aplikasi sangat bagus sangatla marah saya hapu...,negative,1
1,sangat sekali teman teman,positive,5
2,it is just so useful for me,positive,5
3,buat pintar umat manusia,positive,5
4,nice,positive,5
...,...,...,...
195,good,positive,5
196,amat sangat bahagia temu chatgpt rupa temu lua...,positive,5
197,kecewa bisa edit,negative,2
198,sungguh sangat bantu jawab layan tepat sayang ...,positive,5


In [21]:

df['sentiment_second_scheme'] = df['score'].apply(lambda x: 'negative' if x in [1, 2] else ('neutral' if x == 3 else 'positive'))

C:\Users\Alharidt\AppData\Local\Temp\ipykernel_3368\1327490831.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment_second_scheme'] = df['score'].apply(lambda x: 'negative' if x in [1, 2] else ('neutral' if x == 3 else 'positive'))


In [22]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['sentiment_encoded'] = label_encoder.fit_transform(df['sentiment_second_scheme'])




C:\Users\Alharidt\AppData\Local\Temp\ipykernel_3368\1314380004.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment_encoded'] = label_encoder.fit_transform(df['sentiment_second_scheme'])


In [23]:
X = df['contentp_clean']  # Text data
y_multi = df['sentiment_encoded']  # Multi-class labels ("positive," "negative," "neutral")

X_train, X_test, y_multi_train, y_multi_test = train_test_split(
    X, y_multi, test_size=0.2, random_state=42
)

In [24]:
# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Multi-class Classification with Logistic Regression
logistic_classifier = LogisticRegression(max_iter=1000)
logistic_classifier.fit(X_train_tfidf, y_multi_train)

# Predict multi-class sentiment using Logistic Regression
multi_predictions_logistic = logistic_classifier.predict(X_test_tfidf)

# Evaluate multi-class classification model (Logistic Regression)
print("Multi-class Classification Report (Logistic Regression):")
print(classification_report(y_multi_test, multi_predictions_logistic))

Multi-class Classification Report (Logistic Regression):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         1
           2       0.93      1.00      0.96        37

    accuracy                           0.93        40
   macro avg       0.31      0.33      0.32        40
weighted avg       0.86      0.93      0.89        40



C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [25]:
# Multi-class Classification with Random Forest and GridSearchCV
rf_classifier = RandomForestClassifier()
param_grid_rf = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20]}
grid_search_rf = GridSearchCV(rf_classifier, param_grid_rf, cv=3, scoring='accuracy')
grid_search_rf.fit(X_train_tfidf, y_multi_train)
best_rf_model = grid_search_rf.best_estimator_

# Multi-class Classification with XGBoost and GridSearchCV
xgb_classifier = XGBClassifier()
param_grid_xgb = {'n_estimators': [100, 200, 300], 'max_depth': [3, 4, 5], 'learning_rate': [0.01, 0.1, 0.2]}
grid_search_xgb = GridSearchCV(xgb_classifier, param_grid_xgb, cv=3, scoring='accuracy')
grid_search_xgb.fit(X_train_tfidf, y_multi_train)
best_xgb_model = grid_search_xgb.best_estimator_



C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Alharidt\AppData\Roaming\Python\Pyt

In [26]:
# Evaluate binary classification model (Random Forest)
multi_predictions_rf = best_rf_model.predict(X_test_tfidf)
multi_predictions_xgb = best_xgb_model.predict(X_test_tfidf)

print("Multi Classification Report (Random Forest):")
print(classification_report(y_multi_test, multi_predictions_rf))

# Evaluate binary classification model (XGBoost)
print("Multi Classification Report (XGBoost):")
print(classification_report(y_multi_test, multi_predictions_xgb))

Multi Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         1
           2       0.92      0.95      0.93        37

    accuracy                           0.88        40
   macro avg       0.31      0.32      0.31        40
weighted avg       0.85      0.88      0.86        40

Multi Classification Report (XGBoost):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         1
           2       0.92      0.95      0.93        37

    accuracy                           0.88        40
   macro avg       0.31      0.32      0.31        40
weighted avg       0.85      0.88      0.86        40



C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alharidt\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [27]:
def predict_sentiment_multiclass(text):
    # Preprocess the input text (e.g., remove punctuation, lowercase, etc.)
    preprocessed_text = text.lower()  # You can add more pre-processing steps as needed

    # Vectorize the preprocessed text using the TF-IDF vectorizer
    text_vectorized = tfidf_vectorizer.transform([preprocessed_text])

    # Predict sentiment using all three models
    rf_prediction = best_rf_model.predict(text_vectorized)[0]
    xgb_prediction = best_xgb_model.predict(text_vectorized)[0]
    logistic_prediction = logistic_classifier.predict(text_vectorized)[0]

    # Map numeric labels to sentiment labels
    sentiment_labels = {0: "negative", 1: "neutral", 2: "positive"}

    return {
        "Random Forest Prediction": sentiment_labels[rf_prediction],
        "XGBoost Prediction": sentiment_labels[xgb_prediction],
        "Logistic Regression Prediction": sentiment_labels[logistic_prediction],
    }

# Example usage:
input_text = "biasa saja"
sentiment_prediction = predict_sentiment_multiclass(input_text)
print(sentiment_prediction)

{'Random Forest Prediction': 'positive', 'XGBoost Prediction': 'positive', 'Logistic Regression Prediction': 'positive'}


In [28]:
predicted_sentiments_xgb = label_encoder.inverse_transform(multi_predictions_xgb)

# Create a DataFrame to store the test data and predictions
results_df = pd.DataFrame({'Text': X_test, 'True Sentiment': label_encoder.inverse_transform(y_multi_test),
                            'XGBoost Prediction': predicted_sentiments_xgb})

In [29]:
results_df[results_df['XGBoost Prediction'] == 'neutral']

,Text,True Sentiment,XGBoost Prediction
